# SPR 2026 - SGDClassifier v3 (RandomizedSearch)

**Score baseline (v2):** 0.75019

**Melhorias:**
- RandomizedSearchCV para tuning
- Class weights balanceados
- Threshold tuning por classe
- SMOTE para classes 5/6 (opcional)
- Flag para remover classe 2

---
**CONFIGURACAO KAGGLE:** Internet OFF
---

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import loguniform
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - SGDClassifier v3 (RandomizedSearch)")
print("="*60)

SEED = 42
DATA_DIR = '/kaggle/input/competitions/spr-2026-mammography-report-classification'
np.random.seed(SEED)

# FLAGS - AJUSTE AQUI
REMOVE_CLASS_2 = False
USE_SMOTE = False
USE_THRESHOLD_TUNING = True
N_SEARCH_ITER = 30

# Dados
print("\n[1/5] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"Train: {train.shape} | Test: {test.shape}")

# Auto-detectar colunas
def find_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
        for col in df.columns:
            if col.lower() == c.lower():
                return col
    return None

TEXT_COL = find_col(train, ['report', 'text', 'laudo', 'texto', 'content'])
LABEL_COL = find_col(train, ['target', 'label', 'birads', 'classe', 'class'])
ID_COL = find_col(test, ['ID', 'id', 'Id', 'index', 'idx'])
print(f"Colunas: texto={TEXT_COL}, label={LABEL_COL}, id={ID_COL}")

if REMOVE_CLASS_2:
    train = train[train[LABEL_COL] != 2].reset_index(drop=True)
    print(f"Sem classe 2: {train.shape}")

# TF-IDF
print("\n[2/5] TF-IDF...")
tfidf = TfidfVectorizer(max_features=15000, ngram_range=(1,2), min_df=2, max_df=0.95, sublinear_tf=True)
X_train = tfidf.fit_transform(train[TEXT_COL])
X_test = tfidf.transform(test[TEXT_COL])
y_train = train[LABEL_COL].values
print(f"Shape: {X_train.shape}")

# SMOTE opcional
if USE_SMOTE:
    try:
        from imblearn.over_sampling import SMOTE
        smote = SMOTE(sampling_strategy={5:500, 6:500}, random_state=SEED, k_neighbors=3)
        X_train, y_train = smote.fit_resample(X_train, y_train)
        print(f"SMOTE: {X_train.shape}")
    except ImportError:
        print("imblearn nao disponivel")

# RandomizedSearch
print(f"\n[3/5] RandomizedSearchCV ({N_SEARCH_ITER} iter)...")
param_dist = {
    'alpha': loguniform(1e-5, 1e-2),
    'loss': ['log_loss', 'modified_huber', 'hinge'],
    'penalty': ['l2', 'l1', 'elasticnet'],
    'l1_ratio': [0.15, 0.3, 0.5, 0.7],
    'max_iter': [1000, 2000],
    'learning_rate': ['optimal', 'adaptive', 'constant'],
    'eta0': loguniform(1e-3, 1e-1),
}
base = SGDClassifier(class_weight='balanced', random_state=SEED, n_jobs=-1, early_stopping=True, validation_fraction=0.1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
search = RandomizedSearchCV(base, param_dist, n_iter=N_SEARCH_ITER, cv=cv, scoring='f1_macro', n_jobs=-1, random_state=SEED, verbose=1)
search.fit(X_train, y_train)
print(f"Best params: {search.best_params_}")
print(f"Best F1-macro: {search.best_score_:.4f}")

best_model = search.best_estimator_

# Threshold tuning
if USE_THRESHOLD_TUNING and hasattr(best_model, 'predict_proba'):
    print("\n[4/5] Threshold tuning...")
    proba = best_model.predict_proba(X_test)
    classes = best_model.classes_
    thresholds = {0:0.5, 1:0.5, 2:0.5, 3:0.5, 4:0.5, 5:0.35, 6:0.35}
    preds = []
    for i in range(len(proba)):
        adj = proba[i].copy()
        for j,c in enumerate(classes):
            if c in thresholds:
                adj[j] *= (0.5/thresholds[c])
        preds.append(classes[np.argmax(adj)])
    predictions = np.array(preds)
else:
    print("\n[4/5] Predicao padrao...")
    predictions = best_model.predict(X_test)

# Submissao
print("\n[5/5] Gerando submissao...")

# Ler sample_submission para colunas corretas
sample_path = f'{DATA_DIR}/sample_submission.csv'
if os.path.exists(sample_path):
    sample_sub = pd.read_csv(sample_path)
    SUB_ID = sample_sub.columns[0]
    SUB_LABEL = sample_sub.columns[1]
else:
    SUB_ID = ID_COL
    SUB_LABEL = LABEL_COL

submission = pd.DataFrame({SUB_ID: test[ID_COL], SUB_LABEL: predictions})
submission.to_csv('/kaggle/working/submission.csv', index=False)
print("="*60)
print("CONCLUIDO!")
print("="*60)
print("\nDistribuicao:")
print(submission[SUB_LABEL].value_counts().sort_index())